In [ ]:
import pickle
import numpy as np
import pandas as pd

In [ ]:
customer_data = pd.read_pickle("../input/stock-prices-5y/customer_data.pkl")

In [ ]:
customer_data.keys()

# Generate Rating Dataframe

In [ ]:
# Use the customer's holdings as their ratings 

ratingList = []
for customer in customer_data.keys():
    tmp_data = customer_data[customer]
    for stock,rating in zip(tmp_data['stocks'],tmp_data['positions']):
        ratingList.append([customer,stock,rating])

In [ ]:
df_rating = pd.DataFrame(ratingList,columns = ['customerID','stockID','rating'])

In [ ]:
df_rating.head()

In [ ]:
df_rating.stockID.unique()

In [ ]:
# set a dictionary, stock Name = id
stock2id = {}
for id, stock in enumerate(df_rating.stockID.unique()):
    stock2id[stock]=id

In [ ]:
df_rating.stockID = df_rating.stockID.apply(lambda x :stock2id[x])

In [ ]:
df_rating.head()

# ALS with Spark

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.mllib.recommendation import ALS,MatrixFactorizationModel
from pyspark.sql.types import DoubleType, StringType

In [ ]:
# Create Spark Context
sc = pyspark.SparkContext.getOrCreate()

sqlContext = SQLContext(sc)

In [ ]:
sc

In [ ]:
# after crate a framework, then create Spark Dataframe
df_spark = sqlContext.createDataFrame(df_rating)
df_spark.printSchema()

In [ ]:
df_spark.head(5)

In [ ]:
# Train ALS Model
rank = 5 
# hidden factors/latent factors : what are the factors are you using to predict?
numIterations = 10 
# iterations is the number of iterations of ALS to run.
model = ALS.train(df_spark,rank,numIterations)

In [ ]:
display('features for stock one',model.productFeatures().first()) # 只抽取第一个
display('features for user one', model.userFeatures().first()) 

In [ ]:
# For Product X, find N users to sell to
stockForUser = model.recommendUsers(222,10)
print("Top 10 stock for user 222:\n")        
display(stockForUser)

In [ ]:
# For user y find N products to promote
userForStock = model.recommendProducts(100,10)
print('Top 10 Users to recommend for stock 100')
display(userForStock)

In [ ]:
# Predict Ratings
ratingPredict = model.predict(222,100)
print("Prediction of rate user 222 to stock 100")
display(ratingPredict)

In [ ]:
stockFeatures = model.productFeatures()
stockFeatures.collect()